In [3]:
pip install pytorch-lightning


  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/831.6 kB ? eta -:--:--
   --------------------------------------- 831.6/831.6 kB 35.6 MB/s eta 0:00:00
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
   ---------------------------------------- 0.0/983.2 kB ? eta -:--:--
   --------------------------------------- 983.2/983.2 kB 45.1 MB/s eta 0:00:00

   ------- --------------------------------  2/11 [lightning-utilities]
   -------------- -------------------------  4/11 [attrs]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]
   ----------------------------- ----------  8/11 [torchmetrics]

In [5]:
pip install pytorch-metric-learning


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install xgboost


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ------------- -------------------------- 23.9/72.0 MB 116.2 MB/s eta 0:00:01
   -------------------------- ------------- 48.0/72.0 MB 117.5 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 117.5 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 102.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
# ================================================================
# 0) IMPORTS
# ================================================================
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import pytorch_lightning as pl
from pytorch_metric_learning.losses import TripletMarginLoss
from pytorch_metric_learning.miners import TripletMarginMiner
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [ ]:
# ================================================================
# 1) DATA TRANSFORMS + DATASET (YA SEPARADO)
# ================================================================
DATASET_ROOT = r"D:\DEEp\deep\dataset_split"

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(DATASET_ROOT, "train"), transform=train_transform)
val_dataset   = datasets.ImageFolder(os.path.join(DATASET_ROOT, "val"),   transform=test_transform)
test_dataset  = datasets.ImageFolder(os.path.join(DATASET_ROOT, "test"),  transform=test_transform)

class_names = train_dataset.classes
num_classes = len(class_names)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Clases detectadas:", class_names)

Clases detectadas: ['Abutilon Indicum', 'Aloe barbadensis miller', 'Calotropis gigantea', 'Canna indica', 'Cissus quadrangularis', 'Curcuma longa', 'Eclipta prostrate', 'Eichhornia Crassipes', 'Hibiscus Rosasinensis', 'Ixora coccinea', 'Justica adhatoda', 'Murraya koenigii', 'Ocimum tenuiflorum', 'Ouretlanata', 'Phyllanthus amarus', 'Ricinus communis', 'Senna Atriculata', 'Sesbania grandiflora', 'Trifolium Repens', 'Ziziphus mauritiana']


In [10]:
# ================================================================
# 2) SIAMESE NETWORK (EMBEDDING)
# ================================================================
class SiameseBackbone(nn.Module):
    def __init__(self, embedding_size=128):
        super().__init__()
        self.backbone = models.resnet50(weights="DEFAULT")
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, embedding_size)

    def forward(self, x):
        return self.backbone(x)

siamese = SiameseBackbone().to(device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\I21326/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 119MB/s]


In [11]:
# ================================================================
# 3) PYTORCH LIGHTNING TRAINER PARA TRIPLET LOSS
# ================================================================
class TripletLightning(pl.LightningModule):
    def __init__(self, model, lr=1e-4, margin=0.2):
        super().__init__()
        self.model = model
        self.miner = TripletMarginMiner(margin=margin, type_of_triplets="semihard")
        self.loss_func = TripletMarginLoss(margin=margin)
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        embeddings = self(imgs)

        hard_pairs = self.miner(embeddings, labels)
        loss = self.loss_func(embeddings, labels, hard_pairs)

        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

triplet_trainer = pl.Trainer(max_epochs=5, accelerator=device, log_every_n_steps=20)
triplet_trainer.fit(TripletLightning(siamese), train_loader)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\I21326\.conda\envs\VS\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA RTX 6000 Ada Generation') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')`

RuntimeError: Please call `iter(combined_loader)` first.

In [ ]:
# ================================================================
# 4) CLASIFICADOR FC CONGELANDO EL BACKBONE
# ================================================================
class Classifier(nn.Module):
    def __init__(self, backbone, emb_size, num_classes):
        super().__init__()
        for p in backbone.parameters():
            p.requires_grad = False

        self.backbone = backbone
        self.fc = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        with torch.no_grad():
            emb = self.backbone(x)
        return self.fc(emb)

classifier = Classifier(siamese, 128, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.fc.parameters(), lr=1e-3)

# -------- Entrenamiento del clasificador --------
for epoch in range(8):
    classifier.train()
    running_loss = 0
    for img, lbl in train_loader:
        img, lbl = img.to(device), lbl.to(device)

        optimizer.zero_grad()
        out = classifier(img)
        loss = criterion(out, lbl)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
# ================================================================
# 5) EVALUACIÓN DEL CLASIFICADOR FC
# ================================================================
def evaluate(model, loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            p = out.argmax(1).cpu().numpy()
            preds.extend(p)
            labels.extend(y.numpy())

    acc = accuracy_score(labels, preds)
    f1  = f1_score(labels, preds, average="macro")
    return acc, f1, labels, preds

acc, f1, y_true, y_pred = evaluate(classifier, test_loader)

print("Accuracy Clasificador FC:", acc)
print("F1-score:", f1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusión - FC")
plt.xlabel("Predicción")
plt.ylabel("Verdad")
plt.show()

In [ ]:
# ================================================================
# 6) EXTRAER EMBEDDINGS (PARA XGBOOST)
# ================================================================
def extract_embeddings(model, loader):
    model.eval()
    all_emb, all_lbl = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            emb = model(x).cpu().numpy()
            all_emb.append(emb)
            all_lbl.append(y.numpy())

    return np.vstack(all_emb), np.hstack(all_lbl)

train_emb, train_lbl = extract_embeddings(siamese, train_loader)
test_emb,  test_lbl  = extract_embeddings(siamese, test_loader)

print("Embeddings shape:", train_emb.shape)

In [ ]:
# ================================================================
# 7) XGBOOST PARA CLASIFICAR EMBEDDINGS
# ================================================================
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric='mlogloss'
)

xgb.fit(train_emb, train_lbl)
xgb_preds = xgb.predict(test_emb)

acc_xgb = accuracy_score(test_lbl, xgb_preds)
f1_xgb  = f1_score(test_lbl, xgb_preds, average="macro")

print("🎯 XGBoost Accuracy:", acc_xgb)
print("🔥 XGBoost F1-score:", f1_xgb)


In [ ]:
# ================================================================
# 8) MATRIZ DE CONFUSIÓN XGBOOST
# ================================================================
cm = confusion_matrix(test_lbl, xgb_preds)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Greens", xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusión - XGBoost")
plt.xlabel("Predicción")
plt.ylabel("Verdad")
plt.show()